In [17]:
import numpy as np
import pandas as pd
import torch
import inspect

In [18]:
df = pd.read_csv(r"bitcoin_2010-07-29_2025-04-25.csv")

In [19]:
close_price = df[["Close"]].iloc[::-1].reset_index(drop=True)
close_price.head(10)

,Close
0,0.0699
1,0.0627
2,0.0679
3,0.0611
4,0.0600
5,0.0600
6,0.0570
7,0.0610
8,0.0623
9,0.0590


In [20]:
# Split training and testing data
train_size = int(len(close_price) * 0.8)
train_data_bitcoin = close_price[:train_size]
test_data_bitcoin = close_price[train_size:]

In [21]:
train_data_bitcoin.tail(10)

,Close
4297,39699.033959
4298,36568.759600
4299,36033.063400
4300,35552.564000
4301,33933.542810
4302,30363.865617
4303,31024.703300
4304,28984.535800
4305,29130.038600
4306,29250.004000


In [22]:
test_data_bitcoin.head(10)

,Close
4307,30123.775600
4308,31333.765617
4309,29880.548164
4310,30452.624328
4311,28760.153900
4312,30376.331479
4313,29234.032910
4314,29449.985560
4315,30366.042221
4316,29114.081131


In [23]:
test_data_bitcoin.to_csv("TEST_bitcoin.csv", index=False)
train_data_bitcoin.to_csv("TRAIN_bitcoin.csv", index=False)
# Save the training and testing data to CSV files

In [24]:
def preroll_timeseries_df_named(df, history_len, predict_len, stride=1):
    """
    Rolls a univariate time series into overlapping windows.

    Args:
        df (pd.DataFrame): Input time series [T, 1].
        history_len (int): Number of historical time steps.
        predict_len (int): Number of future time steps.
        stride (int): Sliding step between windows.

    Returns:
        pd.DataFrame: Rolled matrix [history+predict rows, N samples columns]
    """
    # Try to infer variable name
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    df_name = next((name for name, val in callers_local_vars if val is df), "df")

    # Prepare
    data = df.values.squeeze().astype('float32')  # [T]
    total_len = history_len + predict_len
    sequences = []

    # Sliding window
    for i in range(0, len(data) - total_len + 1, stride):
        window = data[i:i + total_len].reshape(-1, 1)  # [T, 1]
        sequences.append(window)

    # Stack windows side-by-side
    rolled_array = np.concatenate(sequences, axis=1)  # [T, N]
    ROLLED_df = pd.DataFrame(rolled_array)

    # Save
    filename = f"ROLLED_{df_name}.csv"
    ROLLED_df.to_csv(filename, index=False)

    print(f"✅ Saved: {filename}")
    print(f"📐 ROLLED_{df_name}.shape = {ROLLED_df.shape}")
    return ROLLED_df


In [25]:
preroll_timeseries_df_named(train_data_bitcoin, history_len=50, predict_len=20, stride=1)

✅ Saved: ROLLED_train_data_bitcoin.csv
📐 ROLLED_train_data_bitcoin.shape = (70, 4238)


,0,1,2,3,4,5,6,7,8,9,...,4228,4229,4230,4231,4232,4233,4234,4235,4236,4237
0,0.0699,0.0627,0.0679,0.0611,0.0600,0.0600,0.0570,0.0610,0.0623,0.0590,...,38296.164062,39219.476562,39033.535156,37714.484375,43181.144531,44359.871094,43938.476562,42450.882812,39120.042969,39409.210938
1,0.0627,0.0679,0.0611,0.0600,0.0600,0.0570,0.0610,0.0623,0.0590,0.0609,...,39219.476562,39033.535156,37714.484375,43181.144531,44359.871094,43938.476562,42450.882812,39120.042969,39409.210938,38385.183594
2,0.0679,0.0611,0.0600,0.0600,0.0570,0.0610,0.0623,0.0590,0.0609,0.0710,...,39033.535156,37714.484375,43181.144531,44359.871094,43938.476562,42450.882812,39120.042969,39409.210938,38385.183594,37991.878906
3,0.0611,0.0600,0.0600,0.0570,0.0610,0.0623,0.0590,0.0609,0.0710,0.0700,...,37714.484375,43181.144531,44359.871094,43938.476562,42450.882812,39120.042969,39409.210938,38385.183594,37991.878906,38724.820312
4,0.0600,0.0600,0.0570,0.0610,0.0623,0.0590,0.0609,0.0710,0.0700,0.0670,...,43181.144531,44359.871094,43938.476562,42450.882812,39120.042969,39409.210938,38385.183594,37991.878906,38724.820312,41955.066406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.0614,0.0611,0.0613,0.0614,0.0628,0.0670,0.0869,0.0938,0.0965,0.0950,...,37660.453125,38467.093750,38503.910156,37732.300781,39699.035156,36568.757812,36033.062500,35552.562500,33933.542969,30363.865234
66,0.0611,0.0613,0.0614,0.0628,0.0670,0.0869,0.0938,0.0965,0.0950,0.0949,...,38467.093750,38503.910156,37732.300781,39699.035156,36568.757812,36033.062500,35552.562500,33933.542969,30363.865234,31024.703125
67,0.0613,0.0614,0.0628,0.0670,0.0869,0.0938,0.0965,0.0950,0.0949,0.1050,...,38503.910156,37732.300781,39699.035156,36568.757812,36033.062500,35552.562500,33933.542969,30363.865234,31024.703125,28984.535156
68,0.0614,0.0628,0.0670,0.0869,0.0938,0.0965,0.0950,0.0949,0.1050,0.1020,...,37732.300781,39699.035156,36568.757812,36033.062500,35552.562500,33933.542969,30363.865234,31024.703125,28984.535156,29130.039062


In [26]:
preroll_timeseries_df_named(test_data_bitcoin, history_len=50, predict_len=20, stride=1)


✅ Saved: ROLLED_test_data_bitcoin.csv
📐 ROLLED_test_data_bitcoin.shape = (70, 1008)


,0,1,2,3,4,5,6,7,8,9,...,998,999,1000,1001,1002,1003,1004,1005,1006,1007
0,30123.775391,31333.765625,29880.548828,30452.625000,28760.154297,30376.332031,29234.033203,29449.986328,30366.042969,29114.082031,...,96594.390625,96586.851562,96503.929688,96544.968750,96515.218750,97372.046875,95783.601562,97799.929688,96551.640625,97483.359375
1,31333.765625,29880.548828,30452.625000,28760.154297,30376.332031,29234.033203,29449.986328,30366.042969,29114.082031,29674.111328,...,96586.851562,96503.929688,96544.968750,96515.218750,97372.046875,95783.601562,97799.929688,96551.640625,97483.359375,97551.710938
2,29880.548828,30452.625000,28760.154297,30376.332031,29234.033203,29449.986328,30366.042969,29114.082031,29674.111328,29557.533203,...,96503.929688,96544.968750,96515.218750,97372.046875,95783.601562,97799.929688,96551.640625,97483.359375,97551.710938,96126.976562
3,30452.625000,28760.154297,30376.332031,29234.033203,29449.986328,30366.042969,29114.082031,29674.111328,29557.533203,29353.619141,...,96544.968750,96515.218750,97372.046875,95783.601562,97799.929688,96551.640625,97483.359375,97551.710938,96126.976562,95811.351562
4,28760.154297,30376.332031,29234.033203,29449.986328,30366.042969,29114.082031,29674.111328,29557.533203,29353.619141,28606.474609,...,96515.218750,97372.046875,95783.601562,97799.929688,96551.640625,97483.359375,97551.710938,96126.976562,95811.351562,95532.968750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,22440.037109,23407.345703,23244.574219,23180.087891,22701.085938,22471.316406,22600.785156,21340.466797,21206.714844,22943.121094,...,83520.031250,85288.460938,83693.101562,84495.070312,83677.648438,84051.960938,84931.218750,84464.640625,85127.562500,85095.421875
66,23407.345703,23244.574219,23180.087891,22701.085938,22471.316406,22600.785156,21340.466797,21206.714844,22943.121094,23847.013672,...,85288.460938,83693.101562,84495.070312,83677.648438,84051.960938,84931.218750,84464.640625,85127.562500,85095.421875,87525.710938
67,23244.574219,23180.087891,22701.085938,22471.316406,22600.785156,21340.466797,21206.714844,22943.121094,23847.013672,23746.273438,...,83693.101562,84495.070312,83677.648438,84051.960938,84931.218750,84464.640625,85127.562500,85095.421875,87525.710938,93375.156250
68,23180.087891,22701.085938,22471.316406,22600.785156,21340.466797,21206.714844,22943.121094,23847.013672,23746.273438,23642.244141,...,84495.070312,83677.648438,84051.960938,84931.218750,84464.640625,85127.562500,85095.421875,87525.710938,93375.156250,93565.023438


In [31]:
ROLLED_test_bitcoin = pd.read_csv("ROLLED_train_data_bitcoin.csv")

ROLLED_test_bitcoin.describe()

,0,1,2,3,4,5,6,7,8,9,...,4228,4229,4230,4231,4232,4233,4234,4235,4236,4237
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,...,70.000000,70.000000,70.000000,70.000000,70.00000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,0.063257,0.063216,0.063561,0.063931,0.064437,0.064937,0.065436,0.066121,0.066707,0.067317,...,41356.446471,41331.769243,41286.249029,41236.520843,41182.50740,40999.403400,40808.901014,40595.273271,40404.975514,40263.974957
std,0.002889,0.002812,0.003989,0.005362,0.006614,0.007533,0.008316,0.009504,0.010405,0.011351,...,2760.968715,2796.212328,2856.385947,2925.634456,3024.82188,3279.316708,3463.436089,3719.335859,3956.249957,4172.672838
min,0.057000,0.057000,0.057000,0.057000,0.057000,0.057000,0.057000,0.059000,0.059000,0.059000,...,37660.453000,36568.758000,36033.062000,35552.562000,33933.54300,30363.865000,30363.865000,28984.535000,28984.535000,28984.535000
25%,0.061425,0.061425,0.061425,0.061425,0.061525,0.061600,0.061650,0.061825,0.061900,0.061900,...,39261.832500,39261.832500,39261.832500,39261.832500,39261.83250,39152.141750,38936.998750,38796.531250,38736.126750,38625.208000
50%,0.062200,0.062200,0.062200,0.062200,0.062250,0.062300,0.062350,0.062550,0.062700,0.062750,...,40692.400000,40692.400000,40692.400000,40692.400000,40692.40000,40543.453500,40502.025000,40430.891500,40277.862500,40277.862500
75%,0.064975,0.064975,0.065000,0.065000,0.065000,0.065225,0.065450,0.065500,0.065875,0.066300,...,43085.384250,43085.384250,43085.384250,43085.384250,43085.38425,42891.660250,42704.253250,42432.348750,42353.417000,42353.417000
max,0.071000,0.071000,0.086900,0.093800,0.096500,0.096500,0.096500,0.105000,0.105000,0.105000,...,47498.664000,47498.664000,47498.664000,47498.664000,47498.66400,47498.664000,47498.664000,47498.664000,47498.664000,47498.664000
